# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# Import libraries
import pandas as pd
from sqlalchemy import create_engine
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import make_scorer, f1_score, classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
import re
import pickle
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('wordnet') 
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
# load dataset
engine = create_engine('sqlite:///Messages.db')
df = pd.read_sql("SELECT * FROM Messages", engine)
X = df['message']
Y = df.drop(['id', 'message', 'original', 'genre'], axis = 1)

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    stop_words = stopwords.words("english")
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [ ]:
In this part I will use Random Forest Classifier model.

In [4]:
pipe = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
# Split data into train and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 42)
# Train pipeline
pipe.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [6]:
# Define function to calculate accuracy:
def class_report(Array1, Array2, col_names):
    class_report = []
    # Evaluate metrics for each set of labels
    for i in range(len(col_names)):
        precision = classification_report(Array1[:,i], Array2[:,i]).split()[-4]
        recall = classification_report(Array1[:,i], Array2[:,i]).split()[-3]
        f1 = classification_report(Array1[:,i], Array2[:,i]).split()[-2]
        class_report.append([precision, recall, f1])
    # Store metrics
    class_report = np.array(class_report)
    data_report = pd.DataFrame(data = class_report, index = col_names, columns = ['Precision', 'Recall', 'F1'])
      
    return data_report

In [7]:
# Predict the output:
Y_train_pred = pipe.predict(X_train)

In [8]:
# Calculate accuracy for training set
col_names = list(Y.columns.values)
print(class_report(np.array(Y_train), Y_train_pred, col_names))

                       Precision Recall    F1
related                     0.99   0.99  0.99
request                     0.99   0.99  0.99
offer                       1.00   1.00  1.00
aid_related                 0.99   0.98  0.98
medical_help                0.99   0.99  0.99
medical_products            0.99   0.99  0.99
search_and_rescue           0.99   0.99  0.99
security                    1.00   1.00  0.99
military                    1.00   1.00  1.00
water                       1.00   1.00  1.00
food                        1.00   1.00  1.00
shelter                     0.99   0.99  0.99
clothing                    1.00   1.00  1.00
money                       1.00   1.00  0.99
missing_people              1.00   1.00  1.00
refugees                    0.99   0.99  0.99
death                       0.99   0.99  0.99
other_aid                   0.98   0.98  0.98
infrastructure_related      0.99   0.99  0.98
transport                   0.99   0.99  0.99
buildings                   0.99  

In [9]:
# Calculate accuracy for test set
Y_test_pred = pipe.predict(X_test)

print(class_report(np.array(Y_test), Y_test_pred, col_names))

                       Precision Recall    F1
related                     0.80   0.82  0.80
request                     0.88   0.89  0.88
offer                       0.99   1.00  0.99
aid_related                 0.75   0.75  0.75
medical_help                0.90   0.92  0.89
medical_products            0.94   0.95  0.94
search_and_rescue           0.97   0.97  0.96
security                    0.96   0.98  0.97
military                    0.96   0.97  0.96
water                       0.95   0.95  0.94
food                        0.93   0.94  0.93
shelter                     0.93   0.93  0.92
clothing                    0.98   0.99  0.98
money                       0.98   0.98  0.97
missing_people              0.98   0.99  0.98
refugees                    0.95   0.97  0.96
death                       0.95   0.96  0.95
other_aid                   0.83   0.87  0.83
infrastructure_related      0.88   0.94  0.91
transport                   0.95   0.96  0.95
buildings                   0.94  

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

Because F1 score is combination index of Precision and Recall, I will use F1 score to find better parameters.

In [10]:
# Define f1 metric for use in grid search scoring object
def f1_metric(y_true, y_pred):
    f1_list = []
    for i in range(np.shape(y_pred)[1]):
        f1 = f1_score(np.array(y_true)[:, i], y_pred[:, i])
        f1_list.append(f1)
        
    avg_f1 = np.mean(f1_list)
    return avg_f1

In [11]:
# Get the parameters of the pipeline:
pipe.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7fd05811fd90>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None,

For Random Forest model, here are parameters which will increase the predictive power of the model:
- max_features: Increasing max_features generally improves the performance of the model as at each node now we have a higher number of options to be considered. 
- n_estimators: This is the number of trees you want to build before taking the maximum voting or averages of predictions.
- min_sample_leaf: If you have built a decision tree before, you can appreciate the importance of minimum sample leaf size. Leaf is the end node of a decision tree. A smaller leaf makes the model more prone to capturing noise in train data.
Therefore, I will use these parameters for tuning:

In [12]:
# Grid search

parameters = {'clf__estimator__max_features': ['auto', 'sqrt'],
              'clf__estimator__n_estimators':[10, 25], 
              'clf__estimator__min_samples_leaf':[1, 2, 5]}

scorer = make_scorer(f1_metric)
cv = GridSearchCV(pipe, param_grid = parameters, scoring = scorer, verbose = 10)

# Find best parameters
T_model = cv.fit(X_train, Y_train)

Fitting 3 folds for each of 12 candidates, totalling 36 fits
[CV] clf__estimator__max_features=auto, clf__estimator__min_samples_leaf=1, clf__estimator__n_estimators=10 


/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


[CV]  clf__estimator__max_features=auto, clf__estimator__min_samples_leaf=1, clf__estimator__n_estimators=10, score=0.23743645216154152, total=  51.9s
[CV] clf__estimator__max_features=auto, clf__estimator__min_samples_leaf=1, clf__estimator__n_estimators=10 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.1min remaining:    0.0s
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


[CV]  clf__estimator__max_features=auto, clf__estimator__min_samples_leaf=1, clf__estimator__n_estimators=10, score=0.24081670332497046, total=  51.9s
[CV] clf__estimator__max_features=auto, clf__estimator__min_samples_leaf=1, clf__estimator__n_estimators=10 


...

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


[CV]  clf__estimator__max_features=sqrt, clf__estimator__min_samples_leaf=5, clf__estimator__n_estimators=25, score=0.14414339496012574, total=  48.6s
[CV] clf__estimator__max_features=sqrt, clf__estimator__min_samples_leaf=5, clf__estimator__n_estimators=25 


/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


[CV]  clf__estimator__max_features=sqrt, clf__estimator__min_samples_leaf=5, clf__estimator__n_estimators=25, score=0.13874037981099863, total=  48.6s


[Parallel(n_jobs=1)]: Done  36 out of  36 | elapsed: 42.0min finished


In [13]:
# Grid search results
T_model.cv_results_

/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py

{'mean_fit_time': array([ 45.68647806,  99.61565026,  27.36395741,  53.51784754,
         22.34025121,  41.28565613,  45.19965625,  99.57091045,
         26.75274611,  53.20077912,  22.00269882,  40.96707479]),
 'std_fit_time': array([ 0.38738712,  0.62588639,  0.4484571 ,  0.48665968,  0.29023603,
         0.19152405,  0.07314988,  0.91693146,  0.38331256,  0.66352762,
         0.34202859,  0.0252415 ]),
 'mean_score_time': array([ 6.49214482,  9.3131365 ,  6.09022053,  8.59177128,  5.7653842 ,
         7.64568194,  6.54241371,  9.2643489 ,  6.06418641,  8.44622207,
         5.77603698,  7.61419296]),
 'std_score_time': array([ 0.06399873,  0.10466689,  0.06687831,  0.08012878,  0.0326346 ,
         0.09599008,  0.08396461,  0.18537006,  0.10644289,  0.03870142,
         0.11224039,  0.0250766 ]),
 'param_clf__estimator__max_features': masked_array(data = ['auto' 'auto' 'auto' 'auto' 'auto' 'auto' 'sqrt' 'sqrt' 'sqrt' 'sqrt'
  'sqrt' 'sqrt'],
              mask = [False False False Fa

In [14]:
#Top mean score
np.max(T_model.cv_results_['mean_test_score'])

0.26291376424904955

In [15]:
# Parameters for top mean score
T_model.best_params_

{'clf__estimator__max_features': 'auto',
 'clf__estimator__min_samples_leaf': 1,
 'clf__estimator__n_estimators': 25}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [16]:
# evaluating metrics for test set
tuned_pred_test = T_model.predict(X_test)

class_report1 = class_report(np.array(Y_test), tuned_pred_test, col_names)

print(class_report1)

                       Precision Recall    F1
related                     0.80   0.82  0.80
request                     0.88   0.89  0.88
offer                       0.99   1.00  0.99
aid_related                 0.78   0.78  0.77
medical_help                0.91   0.92  0.89
medical_products            0.94   0.95  0.94
search_and_rescue           0.96   0.97  0.96
security                    0.96   0.98  0.97
military                    0.96   0.97  0.96
water                       0.95   0.95  0.95
food                        0.93   0.93  0.93
shelter                     0.93   0.94  0.93
clothing                    0.98   0.99  0.98
money                       0.97   0.98  0.97
missing_people              0.99   0.99  0.98
refugees                    0.96   0.97  0.96
death                       0.96   0.96  0.95
other_aid                   0.83   0.87  0.82
infrastructure_related      0.89   0.94  0.91
transport                   0.95   0.96  0.95
buildings                   0.95  

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In this part, I will use K-Nearest Neighbour instead. KNN common hyperparameter is n_neighbors.

In [17]:
# using SVM instead 
pipe2 = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(KNeighborsClassifier()))
])

parameters2 = {'clf__estimator__n_neighbors': list(range(1, 10))}

cv2 = GridSearchCV(pipe2, param_grid = parameters2, scoring = scorer, verbose = 10)

# Find best parameters
T_model2 = cv2.fit(X_train, Y_train)

Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV] clf__estimator__n_neighbors=1 ...................................
[CV]  clf__estimator__n_neighbors=1, score=0.2606111256917721, total= 1.7min


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  4.8min remaining:    0.0s


[CV] clf__estimator__n_neighbors=1 ...................................
[CV]  clf__estimator__n_neighbors=1, score=0.26838467901651075, total= 1.8min
[CV] clf__estimator__n_neighbors=1 ...................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  9.6min remaining:    0.0s


[CV]  clf__estimator__n_neighbors=1, score=0.2582580207793079, total= 1.7min
[CV] clf__estimator__n_neighbors=2 ...................................


...

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


[CV]  clf__estimator__n_neighbors=9, score=0.22187684757292245, total= 2.1min
[CV] clf__estimator__n_neighbors=9 ...................................


/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


[CV]  clf__estimator__n_neighbors=9, score=0.22676257176860584, total= 2.2min


[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed: 154.4min finished


In [18]:
# grid search results
T_model2.cv_results_

/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py

{'mean_fit_time': array([ 11.68767277,   9.10743499,   9.20784616,   9.39911215,
          9.39787086,   9.46107856,   9.36081529,   9.37413168,   9.37441858]),
 'std_fit_time': array([ 3.61072666,  0.04261246,  0.10911793,  0.0717791 ,  0.09359811,
         0.16565664,  0.30058454,  0.06922401,  0.09086074]),
 'mean_score_time': array([  91.61345895,   96.76825674,  103.50570345,  117.21668283,
         117.24851584,  119.80586449,  118.53870471,  120.50011071,
         120.70730511]),
 'std_score_time': array([ 0.51926669,  0.38556702,  0.20687187,  4.39507754,  2.75520475,
         4.24747835,  4.86139882,  1.97319445,  1.84714495]),
 'param_clf__estimator__n_neighbors': masked_array(data = [1 2 3 4 5 6 7 8 9],
              mask = [False False False False False False False False False],
        fill_value = ?),
 'params': [{'clf__estimator__n_neighbors': 1},
  {'clf__estimator__n_neighbors': 2},
  {'clf__estimator__n_neighbors': 3},
  {'clf__estimator__n_neighbors': 4},
  {'clf__es

In [20]:
# Calculate evaluation metrics for test set
tuned_pred_test2 = T_model2.predict(X_test)

class_report2 = class_report(np.array(Y_test), tuned_pred_test2, col_names)

print(class_report2)

                       Precision Recall    F1
related                     0.74   0.73  0.74
request                     0.82   0.81  0.82
offer                       0.99   0.99  0.99
aid_related                 0.64   0.65  0.65
medical_help                0.88   0.89  0.88
medical_products            0.93   0.93  0.93
search_and_rescue           0.95   0.95  0.95
security                    0.96   0.96  0.96
military                    0.95   0.96  0.95
water                       0.91   0.92  0.91
food                        0.87   0.87  0.87
shelter                     0.88   0.89  0.88
clothing                    0.98   0.98  0.98
money                       0.96   0.97  0.96
missing_people              0.98   0.98  0.98
refugees                    0.95   0.95  0.95
death                       0.94   0.94  0.94
other_aid                   0.81   0.80  0.80
infrastructure_related      0.90   0.90  0.90
transport                   0.93   0.93  0.93
buildings                   0.92  

### 9. Export your model as a pickle file

Comparing 2 models, we can see that the first one has higher f1 score, precision and recall. Therefore, I will export T_model.

In [21]:
# Pickle best model
pickle.dump(T_model, open('disaster_model.sav', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.